# **Part I: Creating ESG Factor**

### **Importing Packages**

In [3]:
import pandas as pd
import os
import numpy as np

#Inspecting Factor
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

#Edit Data
from pandas.tseries.offsets import MonthEnd
from functools import reduce #Get Monthly Data

#Regression
import statsmodels.api as sm #Regression

## **1. Importing & Preparing Data**

### **1.1 Stock Returns**

In [4]:
stock_returns = pd.read_excel("__data/Stock_Return_Data_Wide_Format.xlsx", "ReturnTotal")

In [5]:
#Divides Stock Return values by 100 to get decimal values
stock_returns.loc[:, stock_returns.columns != "Date"] = stock_returns.loc[:, stock_returns.columns != "Date"].apply(lambda x: x / 100)

#Set Date column as date
stock_returns["Date"] = pd.to_datetime(stock_returns["Date"])
stock_returns['Date'] = stock_returns["Date"].dt.date

#Set Date column as Index
stock_returns.set_index("Date", inplace=True)

In [6]:
stock_returns.tail()

,A.N,AA.N,AAL.OQ,AAON.OQ,AAP.N,AAPL.OQ,AAT.N,ABBV.N,ABCB.N,ABG.N,...,YETI.N,YOU.N,YUM.N,ZBH.N,ZBRA.OQ,ZD.OQ,ZI.OQ,ZION.OQ,ZTS.N,ZWS.N
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-30,-0.005981,-0.101423,-0.014783,0.163717,-0.103482,0.095553,0.044880,0.063756,0.011011,-0.030629,...,-0.063574,0.113823,-0.036164,-0.055398,-0.010918,-0.044437,0.039902,0.004168,0.022411,-0.061003
2024-07-31,0.092844,-0.169432,-0.060900,0.014787,0.003973,0.054411,0.184987,0.090424,0.239588,0.181463,...,0.083879,0.141101,0.002793,0.025984,0.136795,-0.130245,-0.110415,0.191377,0.041036,0.104082
2024-08-31,0.010750,-0.025335,-0.001880,0.078843,-0.284541,0.032345,0.027526,0.059303,0.012153,-0.087586,...,-0.024909,0.423419,0.020794,0.036911,-0.016544,0.020677,-0.129401,-0.032311,0.019162,0.001645
2024-09-30,0.038903,0.201869,0.058380,0.130130,-0.139484,0.017467,-0.006791,0.005960,0.014766,-0.028701,...,0.017609,0.094090,0.035503,-0.062966,0.072210,-0.004297,0.043478,-0.047215,0.064799,0.108233
2024-10-31,-0.120960,0.041676,0.192171,0.059162,-0.078540,-0.030429,0.008608,0.040698,-0.006411,-0.045056,...,-0.141847,0.109837,-0.061198,-0.009541,0.031459,-0.049116,0.070736,0.102499,-0.082752,0.004452


### **1.2 Stock MCap**

In [7]:
stock_mcap = pd.read_excel("__data/Stock_Return_Data_Wide_Format.xlsx", "MCAP")

In [8]:
stock_mcap["Date"] = pd.to_datetime(stock_mcap["Date"])
stock_mcap['Date'] = stock_mcap["Date"].dt.date

stock_mcap.set_index("Date", inplace=True)

In [9]:
stock_mcap.tail()

,A.N,AA.N,AAL.OQ,AAON.OQ,AAP.N,AAPL.OQ,AAT.N,ABBV.N,ABCB.N,ABG.N,...,YETI.N,YOU.N,YUM.N,ZBH.N,ZBRA.OQ,ZD.OQ,ZI.OQ,ZION.OQ,ZTS.N,ZWS.N
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-30,3.782096e+10,7.142957e+09,7.433504e+09,7.171938e+09,3.775880e+09,3.229664e+12,1.362801e+09,3.028816e+11,3.479551e+09,4.596648e+09,...,3.251849e+09,2.709769e+09,3.730500e+10,2.232770e+10,1.588500e+10,2.539352e+09,4.774940e+09,6.403786e+09,7.910332e+10,5.077287e+09
2024-07-31,4.065669e+10,5.932712e+09,6.987330e+09,7.172094e+09,3.775880e+09,3.376535e+12,1.615110e+09,3.273388e+11,4.207941e+09,5.430770e+09,...,3.524612e+09,3.092120e+09,3.732523e+10,2.267669e+10,1.811451e+10,2.208613e+09,4.149581e+09,7.631480e+09,8.215138e+10,5.548339e+09
2024-08-31,4.106487e+10,8.292718e+09,6.974196e+09,7.737566e+09,2.703842e+09,3.481747e+12,1.659568e+09,3.467509e+11,4.256600e+09,4.907912e+09,...,3.413169e+09,4.231689e+09,3.793478e+10,2.351370e+10,1.781483e+10,2.186480e+09,3.612619e+09,7.319840e+09,8.313033e+10,5.543211e+09
2024-09-30,4.266241e+10,9.966763e+09,7.385836e+09,8.761666e+09,2.326700e+09,3.522211e+12,1.627290e+09,3.488176e+11,4.309091e+09,4.767052e+09,...,3.473271e+09,4.614616e+09,3.928156e+10,2.198427e+10,1.908705e+10,2.177085e+09,3.769689e+09,6.974347e+09,8.851710e+10,6.104267e+09
2024-10-31,3.716350e+10,1.035745e+10,8.805555e+09,9.280021e+09,2.129775e+09,3.414816e+12,1.641298e+09,3.602669e+11,4.281465e+09,4.462807e+09,...,2.986704e+09,5.121471e+09,3.687760e+10,2.128496e+10,1.970202e+10,2.070154e+09,3.794225e+09,7.689846e+09,8.065928e+10,6.126672e+09


### **1.3 ESG Scores**

In [10]:
esg_scores = pd.read_excel("__data/Stock_ESG_Data_Wide_Format.xlsx", "ESG")

In [11]:
#Set Date column as date
esg_scores["Date"] = pd.to_datetime(esg_scores["Date"])
esg_scores['Date'] = esg_scores["Date"].dt.date

#Set Date column as Index
esg_scores.set_index("Date", inplace=True)

esg_scores = esg_scores[esg_scores.index >= pd.to_datetime("2009-01-01").date()]

#Fill missing rows with previous values for esg score
esg_scores.fillna(method='ffill', inplace=True)

<ipython-input-11-d126a9c08feb>:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  esg_scores.fillna(method='ffill', inplace=True)


In [12]:
#The data between returns and ESG doesn't always match! Therefore we only keep matching columns
common_columns_returns = esg_scores.columns.intersection(stock_returns.columns)
esg_scores = esg_scores[common_columns_returns]

In [13]:
esg_scores.to_csv("__data/esg_scores.csv")

### **1.4 Fama-French Data**

In [14]:
ff5 = pd.read_csv("__data/F-F_Research_Data_5_Factors_2x3.csv", skiprows=3, index_col=0)

# Convert the index to datetime
ff5.index = pd.to_datetime(ff5.index, format='%Y%m') + MonthEnd(0)

# Remove any potential whitespace in column names
ff5.columns = ff5.columns.str.strip()

# Convert data to numeric, replacing any non-numeric values with NaN
for col in ff5.columns:
    ff5[col] = pd.to_numeric(ff5[col], errors='coerce')
    ff5[col] = ff5[col] / 100 #Divide by 100 to get actual "returns" as decimals

# Drop observations older than 2009-01-01
ff5 = ff5[ff5.index >= '2009-01-01']

# Create a new "Date" column from the index
ff5["Date"] = ff5.index
ff5['Date'] = ff5["Date"].dt.date

# Sort columns and drop index
ff5 = ff5[['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']]
ff5.reset_index(drop=True, inplace=True)

# Display the first few rows of the resulting dataframe
ff5.head()

<ipython-input-14-bf16f5404d74>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ff5["Date"] = ff5.index
<ipython-input-14-bf16f5404d74>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ff5['Date'] = ff5["Date"].dt.date


,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,2009-01-31,-0.0812,-0.0214,-0.1129,0.0018,-0.0116,0.0000
1,2009-02-28,-0.1010,-0.0133,-0.0695,0.0120,-0.0102,0.0001
2,2009-03-31,0.0895,0.0067,0.0347,-0.0252,-0.0225,0.0002
3,2009-04-30,0.1018,0.0713,0.0536,0.0131,0.0012,0.0001
4,2009-05-31,0.0521,-0.0232,0.0028,-0.0078,-0.0216,0.0000


### **1.5 Momentum Data**

In [15]:
mom = pd.read_csv("__data/F-F_Momentum_Factor.CSV", index_col = 0)

# Convert the index to datetime
mom.index = pd.to_datetime(mom.index, format='%Y%m') + MonthEnd(0)

# Remove any potential whitespace in column names
mom.columns = mom.columns.str.strip()

# Convert data to numeric, replacing any non-numeric values with NaN
for col in mom.columns:
    mom[col] = pd.to_numeric(mom[col], errors='coerce')
    mom[col] = mom[col] / 100 #Divide by 100 to get actual "returns" as decimals

# Drop observations older than 2009-01-01
mom = mom[mom.index >= '2009-01-01']

# Create a new "Date" column from the index
mom["Date"] = mom.index
mom['Date'] = mom["Date"].dt.date

mom["MOM"] = mom["Mom"]

mom = mom[["Date", "MOM"]]
mom.reset_index(drop=True, inplace=True)

# Display the first few rows of the resulting dataframe
mom.head()

,Date,MOM
0,2009-01-31,-0.0218
1,2009-02-28,0.0441
2,2009-03-31,-0.1187
3,2009-04-30,-0.3430
4,2009-05-31,-0.1249


## **2. Calculating ESG Factors**

### **2.1 Functions**

In [29]:
def getESGScorePercentile(esg_score_data, mcap_data, percentile, high = True):
    stock_list = []

    #Loops over each date
    for date in esg_score_data.index:
        esg_scores = esg_score_data.loc[date] #Gets corresponding esg scores
        esg_scores = esg_scores.dropna(axis=0) #Drops any missing values

        #Drops Date column as we don't want to look at this
        esg_scores = esg_scores.drop(columns = ["Date"])

        #Creates Portfolio
        if high:
            percentile_stocks = esg_scores.nlargest(int(len(esg_scores) * percentile / 100)).index.tolist()
            avg_esg_score = esg_scores.nlargest(int(len(esg_scores) * percentile / 100)).mean()
        else:
            percentile_stocks = esg_scores.nsmallest(int(len(esg_scores) * percentile / 100)).index.tolist()
            avg_esg_score = esg_scores.nsmallest(int(len(esg_scores) * percentile / 100)).mean()

        mcap = mcap_data.loc[date, percentile_stocks]
        total_mcap = mcap.sum() / 1000000000
        average_mcap = mcap.mean() / 1000000000

        #Counts stocks in portfolio
        count = len(percentile_stocks)

        #Appends to list
        stock_list.append((date, percentile_stocks, count, total_mcap, average_mcap, avg_esg_score))

    #Creates new dataframe
    stock_list_dataframe = pd.DataFrame(stock_list, columns=["Date", "Stock_List", "Stock_Count", "Stock_MCap_Total", "Stock_MCap_Average", "Avg_ESG_Score"])
    stock_list_dataframe.set_index("Date", inplace = True)

    #Returns dataframe
    return stock_list_dataframe

In [30]:
#Calculates POrtfolio return for a specific date & list of stocks
def getPortfolioReturn(return_data, date, list_stocks):
    returns = return_data.loc[date]
    returns = returns[list_stocks]

    return returns

In [31]:
#Calculates Return history
def calculateReturnHistory(return_data, portfolio_data, column_name_return, column_name_count, column_name_mcap_total, column_name_mcap_average, column_name_avg_esg_score):

    average_returns = []

    #Loops over each date
    for date in portfolio_data.index:

        #Gets list of stocks & count of stocks
        currentStockList = portfolio_data.loc[date]["Stock_List"]
        currentStockCount = portfolio_data.loc[date]["Stock_Count"]
        currentStockMCapTotal = portfolio_data.loc[date]["Stock_MCap_Total"]
        currentStockMCapAverage = portfolio_data.loc[date]["Stock_MCap_Average"]
        currentAvgESGScore = portfolio_data.loc[date]["Avg_ESG_Score"]

        #Gets return of list of stocks at current date
        stock_returns = getPortfolioReturn(return_data, date, currentStockList)

        #Calculates average return (EQUAL WEIGHTED)
        if len(stock_returns) > 0:
            average_portfolio_return = stock_returns.mean()
        else:
            average_portfolio_return = float('nan')

        #Adds return to list
        average_returns.append((date, average_portfolio_return, currentStockCount, currentStockMCapTotal, currentStockMCapAverage, currentAvgESGScore))

    #Returns dataframe
    dataframe = pd.DataFrame(average_returns, columns=["Date", column_name_return, column_name_count, column_name_mcap_total, column_name_mcap_average, column_name_avg_esg_score])

    return dataframe

In [32]:
def getESGFactor(esg_score_data, percentile, min_stocks, mcap_data = stock_mcap):
  #Returns dataframe containing the ESG Portfolios for each date

  highest_stocks = []
  lowest_stocks = []

  highest_stocks = getESGScorePercentile(esg_score_data, mcap_data, percentile, high = True)
  lowest_stocks = getESGScorePercentile(esg_score_data, mcap_data, percentile, high = False)

  #Calculates the Average Return for each Portfolio at each Date
  highest_stocks_average_return = calculateReturnHistory(stock_returns, highest_stocks, "Average_Return_High", "Count_High", "MCap_Total_High", "MCap_Average_High", "Avg_ESG_Score_High")
  lowest_stocks_average_return = calculateReturnHistory(stock_returns, lowest_stocks, "Average_Return_Low", "Count_Low", "MCap_Total_Low", "MCap_Average_Low", "Avg_ESG_Score_Low")

  #Merges Data together to have the data in one dataframe
  return_history = pd.merge(highest_stocks_average_return, lowest_stocks_average_return, on='Date', how='outer')

  #Calculates Factor for each Date
  return_history["ESG_Factor"] = return_history["Average_Return_Low"] - return_history["Average_Return_High"]

  return_history_used = return_history[return_history["Count_High"] > min_stocks].copy() #Only look at diversified portfolios
  #return_history_used = return_history

  return_history_used = return_history_used.reset_index(drop = True)

  return return_history_used

### **2.2 Calculating ESG Factor**

In [33]:
esg_factor = getESGFactor(esg_scores, percentile = 25, min_stocks = 50)

In [34]:
highest_stocks = getESGScorePercentile(esg_scores, stock_mcap, percentile = 25, high = True)
lowest_stocks = getESGScorePercentile(esg_scores, stock_mcap, percentile = 25, high = False)

### **2.3 Calculating Residual ESG Factor**

#### **Merge Data**

In [35]:
esg_factor_subsetted = esg_factor[["Date", "ESG_Factor"]]

data = reduce(lambda left, right: pd.merge(left, right, on = "Date"), [ff5, mom, esg_factor_subsetted])

In [36]:
data.tail()

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF,MOM,ESG_Factor
169,2024-01-31,0.0070,-0.0568,-0.0247,0.0066,-0.0102,0.0047,0.0508,-0.018336
170,2024-02-29,0.0507,-0.0076,-0.0352,-0.0198,-0.0216,0.0042,0.0498,0.013514
171,2024-03-31,0.0283,-0.0118,0.0421,0.0147,0.0119,0.0043,-0.0040,-0.017662
172,2024-04-30,-0.0467,-0.0256,-0.0052,0.0148,-0.0030,0.0047,-0.0042,-0.019939
173,2024-05-31,0.0434,0.0076,-0.0166,0.0298,-0.0307,0.0044,-0.0002,0.022154


#### **Run Residual Regression**

In [37]:
Y = data["ESG_Factor"]
X = data[["Mkt-RF", "SMB", "HML", "RMW", "CMA", "MOM"]]
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit(cov_type="HC3")

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             ESG_Factor   R-squared:                       0.662
Model:                            OLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                     49.55
Date:                Mon, 20 Jan 2025   Prob (F-statistic):           1.25e-34
Time:                        21:20:43   Log-Likelihood:                 559.05
No. Observations:                 174   AIC:                            -1104.
Df Residuals:                     167   BIC:                            -1082.
Df Model:                           6                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0047      0.001      5.571      0.0

In [38]:
data["ESG_Factor_Residual"] = model.params["const"] + (Y - model.predict(X))
#data["ESG_Factor_Residual"] = (Y - model.predict(X))

#### **Merge Data back to esg_factor**

In [39]:
data_subsetted = data[["Date", "ESG_Factor_Residual"]]

In [40]:
esg_factor_final = reduce(lambda left, right: pd.merge(left, right, on = "Date"), [esg_factor, data_subsetted])

In [41]:
esg_factor_final.head()

,Date,Average_Return_High,Count_High,MCap_Total_High,MCap_Average_High,Avg_ESG_Score_High,Average_Return_Low,Count_Low,MCap_Total_Low,MCap_Average_Low,Avg_ESG_Score_Low,ESG_Factor,ESG_Factor_Residual
0,2009-12-31,0.038894,154,6110.540896,39.678837,70.750223,0.057903,154,908.712776,6.017965,18.733658,0.019009,-0.010181
1,2010-01-31,-0.033412,156,5988.273673,38.386370,70.962015,-0.030241,156,912.673663,5.965187,18.795111,0.003171,0.005373
2,2010-02-28,0.043410,156,6176.713433,39.594317,70.951422,0.054302,156,963.517698,6.297501,18.802700,0.010892,0.005079
3,2010-03-31,0.069097,156,6533.051561,41.878536,70.999618,0.070676,156,1005.762128,6.573609,18.795148,0.001579,-0.006843
4,2010-04-30,0.025750,156,6633.206082,42.520552,70.952922,0.045017,156,1022.315223,6.681799,18.983957,0.019267,0.000317


## **3. Exporting Data**

In [42]:
#Exports Data as CSV
esg_factor_final.to_csv("__data/esg_factor.csv", index=False)

highest_stocks.to_csv("__data/high_esg_portfolio.csv", index = True)
lowest_stocks.to_csv("__data/low_esg_portfolio.csv", index = True)